In [1]:
import roslibpy

client = roslibpy.Ros(host='robot-mb-01', port=9090)
client.run()
print('Is ROS connected?', client.is_connected)

Is ROS connected? True


In [2]:
first_person_pose = {}
robot_pose = {}

In [3]:
def set_robot_pose(pose):
    global robot_pose
    robot_pose = { 'x': pose['x'] , 'y': pose['y'] }

In [4]:
def set_first_person_pose(people):
    global first_person_pose
    if len(people) == 1:
        #print('One person spotted')
        first_person_pose = { 'x': people[0]['position']['x'] , 'y': people[0]['position']['y'] }
    elif len(people) > 1:
        print('More than one person spotted -> using position of the highest reliability detection')
        highest_reliability = 0
        highest_index = 0
        for i, person in enumerate(people):
            if person['reliability'] > highest_reliability:
                highest_index = i
                highest_reliability = person['reliability']
        person_pose = { 'x': people[highest_index]['position']['x'] , 'y': people[highest_index]['position']['y'] }

In [5]:
robot_listener = roslibpy.Topic(client, '/robot_pose', 'geometry_msgs/PoseStamped')
robot_listener.subscribe(lambda message: set_robot_pose(message['pose']['position']))

In [6]:
people_listener = roslibpy.Topic(client, '/people', 'people_msgs/People')
people_listener.subscribe(lambda message: set_first_person_pose(message['people']))

In [9]:
robot_pose, first_person_pose

({'x': 0.05000000000000001, 'y': -0.04999999999999999},
 {'x': -1.1153478358238886, 'y': -4.023293544730814})

In [10]:
from math import sqrt
from time import sleep
from IPython.display import display, clear_output

while(True):
    clear_output(wait=True)
    x_sq = pow(robot_pose['x'] - first_person_pose['x'], 2)
    y_sq = pow(robot_pose['y'] - first_person_pose['y'], 2)

    distance = sqrt(x_sq+y_sq)
    display(distance)
    sleep(1)

4.143114443492141

KeyboardInterrupt: 

In [11]:
robot_listener.unsubscribe()
people_listener.unsubscribe()